In [15]:
import numpy as np
import joblib 
import pandas as pd

from src.pipeline_components import DropCols, EnsureCategorical, EnsureNumeric, XGBWithAutoSPW, LogTransform
from src import train_score_pipeline, apply_pipeline_to_new_data, proba_to_score, rating, decision_by_score, build_scoring_df, prepare_X_for_model, build_history_features

In [ ]:
import pandas as pd

df_new = pd.DataFrame([
    {
        "ID": 9001001, "CODE_GENDER": 1, "years": 40, "CNT_CHILDREN": 0, "CNT_FAM_MEMBERS": 2,
        "FLAG_OWN_CAR": 1, "FLAG_OWN_REALTY": 1, "NAME_INCOME_TYPE": "State servant",
        "NAME_EDUCATION_TYPE": "Higher education", "NAME_FAMILY_STATUS": "Married",
        "NAME_HOUSING_TYPE": "House / apartment", "OCCUPATION_TYPE": "Managers",
        "years_employed": 15, "amt_income_month": 22000, "renda_per_capita": 11000,
        "no_formal_employment": 0, "unclassified_occupation": 0,
    },
    {
        "ID": 9001002, "CODE_GENDER": 0, "years": 32, "CNT_CHILDREN": 1, "CNT_FAM_MEMBERS": 3,
        "FLAG_OWN_CAR": 1, "FLAG_OWN_REALTY": 1, "NAME_INCOME_TYPE": "Working",
        "NAME_EDUCATION_TYPE": "Higher education", "NAME_FAMILY_STATUS": "Married",
        "NAME_HOUSING_TYPE": "House / apartment", "OCCUPATION_TYPE": "Core staff",
        "years_employed": 6, "amt_income_month": 9000, "renda_per_capita": 3000,
        "no_formal_employment": 0, "unclassified_occupation": 0,
    },
    {
        "ID": 9001003, "CODE_GENDER": 1, "years": 28, "CNT_CHILDREN": 0, "CNT_FAM_MEMBERS": 1,
        "FLAG_OWN_CAR": 0, "FLAG_OWN_REALTY": 0, "NAME_INCOME_TYPE": "Working",
        "NAME_EDUCATION_TYPE": "Secondary / secondary special", "NAME_FAMILY_STATUS": "Single / not married",
        "NAME_HOUSING_TYPE": "Rented apartment", "OCCUPATION_TYPE": "Sales staff",
        "years_employed": 2, "amt_income_month": 4500, "renda_per_capita": 4500,
        "no_formal_employment": 0, "unclassified_occupation": 0,
    },
    {
        "ID": 9001004, "CODE_GENDER": 0, "years": 45, "CNT_CHILDREN": 2, "CNT_FAM_MEMBERS": 4,
        "FLAG_OWN_CAR": 0, "FLAG_OWN_REALTY": 0, "NAME_INCOME_TYPE": "Working",
        "NAME_EDUCATION_TYPE": "Secondary / secondary special", "NAME_FAMILY_STATUS": "Married",
        "NAME_HOUSING_TYPE": "Municipal apartment", "OCCUPATION_TYPE": "Laborers",
        "years_employed": 1, "amt_income_month": 2800, "renda_per_capita": 700,
        "no_formal_employment": 1, "unclassified_occupation": 1,
    },
    {
        "ID": 9001005, "CODE_GENDER": 1, "years": 21, "CNT_CHILDREN": 2, "CNT_FAM_MEMBERS": 4,
        "FLAG_OWN_CAR": 0, "FLAG_OWN_REALTY": 0, "NAME_INCOME_TYPE": "Unemployed",
        "NAME_EDUCATION_TYPE": "Lower secondary", "NAME_FAMILY_STATUS": "Single / not married",
        "NAME_HOUSING_TYPE": "With parents", "OCCUPATION_TYPE": "Low-skill Laborers",
        "years_employed": 0, "amt_income_month": 1100, "renda_per_capita": 275,
        "no_formal_employment": 1, "unclassified_occupation": 1,
    },
])

In [ ]:
df_record_new = pd.DataFrame([
    {"ID": 9001001, "MONTHS_BALANCE": 0, "STATUS": "C"},
    {"ID": 9001001, "MONTHS_BALANCE": -12, "STATUS": "C"},
    {"ID": 9001001, "MONTHS_BALANCE": -24, "STATUS": "X"},

    {"ID": 9001002, "MONTHS_BALANCE": 0, "STATUS": "X"},
    {"ID": 9001002, "MONTHS_BALANCE": -1, "STATUS": "X"},

    {"ID": 9001003, "MONTHS_BALANCE": 0, "STATUS": "C"},
    {"ID": 9001003, "MONTHS_BALANCE": -1, "STATUS": "0"},
    {"ID": 9001003, "MONTHS_BALANCE": -2, "STATUS": "0"},

    {"ID": 9001004, "MONTHS_BALANCE": 0, "STATUS": "2"},
    {"ID": 9001004, "MONTHS_BALANCE": -1, "STATUS": "1"},
    {"ID": 9001004, "MONTHS_BALANCE": -2, "STATUS": "0"},

    {"ID": 9001005, "MONTHS_BALANCE": 0, "STATUS": "5"},
    {"ID": 9001005, "MONTHS_BALANCE": -1, "STATUS": "4"},
    {"ID": 9001005, "MONTHS_BALANCE": -2, "STATUS": "4"},
])

In [18]:
pipeline = joblib.load('models/credit_pipeline_v3.pkl')
score_params = joblib.load('models/score_params_v3.pkl')

hist_features = build_history_features(df_record_new, window_months=12)

df_scoring = build_scoring_df(df_new, hist_features)
X_new, ids = prepare_X_for_model(df_scoring)
df_scored = apply_pipeline_to_new_data(X_new, pipeline, score_params)
if ids is not None:
    df_scored["ID"] = ids.values
df_scored

,CODE_GENDER,years,CNT_CHILDREN,CNT_FAM_MEMBERS,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,last_status,n_months,last_month,vintage,last_bad,proba_bad,score,rating,decision,ID
0,1,40,0,2,1,1,State servant,Higher education,Married,House / apartment,...,0,2,0,12,-1.0,0.258652,666.066223,B - Bom,Aprovado,9001001
1,0,32,1,3,1,1,Working,Higher education,Married,House / apartment,...,0,2,0,1,-1.0,0.451493,582.596558,C - Regular,Aprovado com Restrição,9001002
2,1,28,0,1,0,0,Working,Secondary / secondary special,Single / not married,Rented apartment,...,0,3,0,2,-1.0,0.266814,661.968506,B - Bom,Aprovado,9001003
3,0,45,2,4,0,0,Working,Secondary / secondary special,Married,Municipal apartment,...,2,3,0,2,0.0,0.982554,300.000000,E - Alto Risco,Reprovado,9001004
4,1,21,2,4,0,0,Unemployed,Lower secondary,Single / not married,With parents,...,5,3,0,2,0.0,0.972506,300.000000,E - Alto Risco,Reprovado,9001005


# 🚀 Aplicação do Pipeline em Novos Dados (Simulação de Produção)

## 🎯 Objetivo

Validar o funcionamento do pipeline treinado em um cenário de “produção”, aplicando o modelo salvo (`.pkl`) em novos clientes e retornando:

- `proba_bad` (probabilidade prevista)
- `score` (escala 300–850)
- `rating` (A–E)
- `decision` (Aprovado / Restrição / Manual / Reprovado)

---

## 📁 Definição do Ambiente e Imports

Nesta etapa foi importadas as funções do módulo `src`:

```python

import numpy as np
import joblib
import pandas as pd

from src.pipeline_components import DropCols, EnsureCategorical, EnsureNumeric, XGBWithAutoSPW, LogTransform
from src import (
    train_score_pipeline,
    apply_pipeline_to_new_data,
    proba_to_score,
    rating,
    decision_by_score,
    build_scoring_df,
    prepare_X_for_model,
    build_history_features
)
```

---

## 🧪 Criação de Novos Clientes (Input)

Foi criado um `df_new` contendo perfis simulados (A, B, C, etc.), com variáveis cadastrais necessárias para o modelo.

---

## 💾 Carregamento do Pipeline Treinado e Parâmetros do Score

O pipeline e os parâmetros da escala do score foram carregados a partir de arquivos `.pkl`:

```python
pipeline = joblib.load("models/credit_pipeline_v3.pkl")
score_params = joblib.load("models/score_params_v3.pkl")
```

---

## 🧱 Construção das Features Históricas (Vintage Features)

O histórico foi transformado em features agregadas:

```python
hist_features = build_history_features(df_record_new, window_months=12)
```

---

## 🔗 Montagem do Dataset para Scoring

Foi feito o merge entre cadastro (`df_new`) e histórico (`hist_features`) para gerar a base completa de scoring.

```python
df_scoring = build_scoring_df(df_new, hist_features)
```

---

## 🧰 Preparação Final para o Modelo

A base foi preparada para inferência, e os IDs foram preservados para reanexar no final:

```python
X_new, ids = prepare_X_for_model(df_scoring)
```

---

## ✅ Aplicação do Pipeline (Inferência)

Aplicação do pipeline em novos dados:

```python
df_scored = apply_pipeline_to_new_data(X_new, pipeline, score_params)

if ids is not None:
    df_scored["ID"] = ids.values

df_scored
```

O output final contém, para cada cliente:

- `proba_bad`
- `score`
- `rating`
- `decision`
- `ID` (reatribuído após a preparação)

---

## ✅ Resultado

Esta etapa valida a execução end-to-end em modo produção, garantindo que:

- o pipeline treinado pode ser carregado via `joblib`
- novas observações podem ser transformadas e avaliadas
- o sistema entrega score e decisão de forma automática
